In [13]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement re (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Antony\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for re


In [15]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from scipy.stats import uniform, randint
import optuna


In [2]:
train = pd.read_csv('data/processed/train_cleaned.csv')
test = pd.read_csv('data/processed/test_cleaned.csv')

In [3]:
drop_columns = ['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin', 'Age','Fare']
drop_test = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Age','Fare']


y_train = train['Survived']
X = train.drop(drop_columns, axis=1)
X_test = test.drop(drop_test, axis=1)
x_train = X.values # Creates an array of the train data
x_test = X_test.values 


## Grid Search CV

Principe : on définit un ensemble de valeurs possibles pour chaque hyperparamètre, et on teste toutes les combinaisons.

Avantage : simple à implémenter, bonne couverture.

Inconvénient : très coûteux en temps de calcul, surtout avec de nombreux paramètres.

In [9]:
param_grid = {
    'n_estimators': [100, 300],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1],
}
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(x_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Score sur validation croisée :", grid_search.best_score_)

Fitting 3 folds for each of 192 candidates, totalling 576 fits


C:\Users\Antony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [13:46:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 300, 'subsample': 0.8}
Score sur validation croisée : 0.8439955106621774


**Meilleurs paramètres selon GridSearch CV:**

| Paramètre | Valeur |
|-----------|--------|
| colsample_bytree | 0.8 |
| gamma | 0 |
| learning_rate | 0.1 |
| max_depth | 9 |
| min_child_weight | 3 |
| n_estimators | 300 |
| subsample | 0.8 |

Using 3-fold CV


In [21]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
    n_estimators= 300,
    max_depth= 9,
    min_child_weight= 3,
    gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(x_train, y_train)

xgb_predictions = gbm.predict(x_test)
predictions_train = gbm.predict(x_train)

PassengerId = test['PassengerId']
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId, 'Survived': xgb_predictions })

# Evaluate the model on the training set
accuracy = accuracy_score(train['Survived'], predictions_train)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_matrix(train['Survived'], predictions_train))
print("Classification Report:\n", classification_report(train['Survived'], predictions_train))

StackingSubmission.to_csv('data/submissions/XGB_GridSearch.csv', index=False)



Accuracy: 0.9001122334455668
Confusion Matrix:
 [[522  27]
 [ 62 280]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.95      0.92       549
           1       0.91      0.82      0.86       342

    accuracy                           0.90       891
   macro avg       0.90      0.88      0.89       891
weighted avg       0.90      0.90      0.90       891



## Random Search
Principe : au lieu de tester toutes les combinaisons, on en teste un nombre fixe choisi aléatoirement dans l’espace des hyperparamètres.

Avantage : souvent plus efficace que grid search pour un budget donné ; explore mieux quand certains hyperparamètres sont plus importants que d'autres.

Inconvénient : moins systématique, résultats variables.

In [11]:
# Grille aléatoire
param_dist = {
    'n_estimators': randint(100, 500),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.7, 0.3),        
    'colsample_bytree': uniform(0.7, 0.3),    
    'gamma': uniform(0, 0.3),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}

# 4. RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,             # nombre d'itérations aléatoires
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# 5. Entraîner
random_search.fit(x_train, y_train)

# 6. Résultats
print("Meilleurs paramètres :", random_search.best_params_)
print("Score sur validation croisée :", random_search.best_score_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\Antony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [13:46:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Meilleurs paramètres : {'colsample_bytree': 0.7582821860536126, 'gamma': 0.021282275099978296, 'learning_rate': 0.1290351481641665, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 242, 'reg_alpha': 0.375582952639944, 'reg_lambda': 0.093981939840869, 'subsample': 0.8734840422988521}
Score sur validation croisée : 0.8372615039281706


**Meilleurs paramètres selon RandomSearch CV**

| Paramètre | Valeur |
|-----------|--------|
| colsample_bytree | 0.758 |
| gamma | 0.021 |
| learning_rate | 0.129 |
| max_depth | 8 |
| min_child_weight | 3 |
| n_estimators | 242 |
| reg_alpha | 0.376 |
| reg_lambda | 0.094 |
| subsample | 0.873 |


In [20]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.129,
    n_estimators= 242,
    max_depth= 8,
    min_child_weight= 3,
    gamma=0.021,                        
    subsample=0.873,
    colsample_bytree=0.758,
    objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(x_train, y_train)

xgb_predictions = gbm.predict(x_test)
predictions_train = gbm.predict(x_train)

PassengerId = test['PassengerId']
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId, 'Survived': xgb_predictions })

# Evaluate the model on the training set
accuracy = accuracy_score(train['Survived'], predictions_train)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_matrix(train['Survived'], predictions_train))
print("Classification Report:\n", classification_report(train['Survived'], predictions_train))

StackingSubmission.to_csv('data/submissions/XGB_RandomSearch.csv', index=False)


Accuracy: 0.9248035914702581
Confusion Matrix:
 [[529  20]
 [ 47 295]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94       549
           1       0.94      0.86      0.90       342

    accuracy                           0.92       891
   macro avg       0.93      0.91      0.92       891
weighted avg       0.93      0.92      0.92       891



## Bayesian Optimization

Principe : on modélise la fonction de performance du modèle en fonction des hyperparamètres (souvent via un Gaussian Process), et on choisit les hyperparamètres à tester de manière intelligente (exploration/exploitation).

Avantage : beaucoup plus efficace quand le coût d'entraînement est élevé.

Inconvénient : plus complexe à mettre en œuvre

In [17]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.3),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    
    model = XGBClassifier(**params)
    
    # Moyenne des scores sur 3 folds
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()
    return score

# Création et lancement de l'étude
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # tu peux ajuster le nombre de trials

# Affichage des meilleurs paramètres
print("Meilleurs hyperparamètres :", study.best_params)
print("Meilleur score CV :", study.best_value)

[I 2025-06-05 13:58:49,493] A new study created in memory with name: no-name-7158c1f3-6c4f-4de5-9aa2-aa1ce92734cc
C:\Users\Antony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [13:58:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Antony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [13:58:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Antony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: 

Meilleurs hyperparamètres : {'n_estimators': 371, 'learning_rate': 0.23139905503070082, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.9663841101536176, 'colsample_bytree': 0.8203379517193834, 'gamma': 0.1320073460167574, 'reg_alpha': 0.36112268337465814, 'reg_lambda': 0.32169472422277506}
Meilleur score CV : 0.8462401795735129


**Meilleurs paramètres selon Bayesian Optimization**

| Paramètre | Valeur |
|-----------|--------|
| n_estimators | 371 |
| learning_rate | 0.231 |
| max_depth | 10 |
| min_child_weight | 2 |
| subsample | 0.966 |
| colsample_bytree | 0.820 |
| gamma | 0.132 |
| reg_alpha | 0.361 |
| reg_lambda | 0.322 |


In [19]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.231,
    n_estimators= 371,
    max_depth= 10,
    min_child_weight= 2,
    gamma=0.132,                        
    subsample=0.873,
    colsample_bytree=0.758,
    objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(x_train, y_train)

xgb_predictions = gbm.predict(x_test)
predictions_train = gbm.predict(x_train)

PassengerId = test['PassengerId']
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId, 'Survived': xgb_predictions })

# Evaluate the model on the training set
accuracy = accuracy_score(train['Survived'], predictions_train)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_matrix(train['Survived'], predictions_train))
print("Classification Report:\n", classification_report(train['Survived'], predictions_train))

StackingSubmission.to_csv('data/submissions/XGB_Bayesian.csv', index=False)

Accuracy: 0.9494949494949495
Confusion Matrix:
 [[539  10]
 [ 35 307]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       549
           1       0.97      0.90      0.93       342

    accuracy                           0.95       891
   macro avg       0.95      0.94      0.95       891
weighted avg       0.95      0.95      0.95       891

